In [6]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [7]:
driver_path = 'C:/chromedriver_win32/chromedriver.exe' 

In [13]:
def m_i(query,driver_path):
    domain = '매일일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
    url_domain = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=news&query='
    news_domain ='&oquery=&tqi=hdwlCdp0YiRss4g9yiCssssstZw-285143&mynews=1&news_office_checked=2385&office_section_code=1&office_type=1&pd=0&photo=0&sort=0' 
    url = url_domain + query + news_domain
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    # 기사 url을 list형태로 반환
    news_list = soup.select('#main_pack > section > div > div.group_news > ul')
    url_list = []
    news_list = news_list[0].find_all('li')
    
    for url in news_list:
        url_list.append(str(url.find_all('a')[0].get('data-url')))
    driver.close()
    
    data_list = []
    head_list = []
    for url in url_list:
        driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
        time.sleep(2)
        driver.get(url)
        time.sleep(2)
        # try문 안에서 각자 언론사에 맞게 수정
        try:
            soup = BeautifulSoup(driver.page_source,'html.parser')
            head_list.append(soup.select('#user-container > div.custom-pc.float-center.max-width-1130 > header > div > div'))
            tags = soup.find(id='article-view-content-div').find_all('p')
            data = [x.text for x in tags]
            data_list.append(' '.join(data))
        except:
            print('None_type')
        driver.close()
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

mi = m_i('lg전자 슬로건',driver_path)

In [14]:
mi.to_csv('매일일보.csv',index=None)

In [15]:
df = pd.DataFrame(mi)
df

,언론사,제목,내용
0,매일일보,"[[그림, 발레, 오페라와 만나는 ‘LG 시그니처’]]","[매일일보 이상래 기자] LG전자가 러시아 푸쉬킨미술관, 미국 아메리칸발레시어터(A..."
1,매일일보,"[[LG전자, 英 아트스쿨 ‘센트럴 세인트 마틴’과 손잡고 LG시그니처 알린다]]",[매일일보 황병준 기자] LG전자가 초프리미엄 가전 ‘LG 시그니처’를 알리기 위해...
2,매일일보,"[[LG전자, 伊 마시밀리아노 푹사스와 협업해 ‘시그니처’ 전시]]",[매일일보 황병준 기자] LG전자가 다음달 6일부터 11일까지 독일 베를린에서 열리...
3,매일일보,"[[[CES2021] LG전자, 세계에 ‘뉴노멀’ 비전 제시…‘집’ 의미 재정의]]",[매일일보 정두용 기자] “소중한 일상은 계속됩니다. LG와 함께 홈 라이프를 편안...
4,매일일보,"[[LG전자, 새해 맞이 ‘한마음 동행 페스타’… 90여종 행사제품 할인]]",[매일일보 이상래 기자] LG전자가 새해를 맞아 31일까지 ‘한마음 동행 페스타’를...
5,매일일보,[[LG전자 ‘시그니처 키친 스위트’ 국내 데뷔]],[매일일보 이한듬 기자] LG전자의 초프리미엄 빌트인 브랜드 ‘시그니처 키친 스위트...
6,매일일보,[[LG전자 ‘G4’ 국내 출고가 82만5천원]],[매일일보 김창성 기자]LG전자가 6개 주요국가에서 전략 스마트폰 ‘LG G4’를 ...
7,매일일보,"[[구본준 LX 회장 홀로서기, '1등경영' 재현하나]]",[매일일보 이재영 기자] LG 고문에서 LX홀딩스 대표이사로 경영일선에 복귀한 구본...
8,매일일보,[[[2020 CEO 평가-전자②] 내년 턴어라운드 목표…이연모 부사장 “시간이 없...,[매일일보 문수호 기자] LG전자는 지난해 11월 이동단말(MC) 사업본부장에 이연...
9,매일일보,"[[LGU+, 경쟁사 5G B2B 강조에도 “B2C가 중심”…속내는?]]",[매일일보 정두용 기자] LG유플러스가 SK텔레콤·KT 등 경쟁사의 5G 전략과 다...
